In [3]:
# ============================================
# Cybersecurity Logs Practice DB (SQLite)
# Creates sec_logs.db with realistic tables:
#   assets, users, roles, permissions, user_roles, role_permissions
#   auth_logs, process_events, dns_logs, network_flows,
#   alerts, iocs, alert_ioc, cases, case_alert, vuln_findings
# Includes indexes and example queries.
# ============================================

import sqlite3
from datetime import datetime, timedelta
import random

DB_PATH = "sec_logs.db"

def connect_db(path: str) -> sqlite3.Connection:
    conn = sqlite3.connect(path)
    # Enforce foreign keys in SQLite
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

def reset_schema(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()
    cur.executescript("""
    -- Drop in FK-safe order
    DROP TABLE IF EXISTS case_alert;
    DROP TABLE IF EXISTS alert_ioc;
    DROP TABLE IF EXISTS vuln_findings;
    DROP TABLE IF EXISTS alerts;
    DROP TABLE IF EXISTS iocs;
    DROP TABLE IF EXISTS network_flows;
    DROP TABLE IF EXISTS dns_logs;
    DROP TABLE IF EXISTS process_events;
    DROP TABLE IF EXISTS auth_logs;
    DROP TABLE IF EXISTS casebook;
    DROP TABLE IF EXISTS role_permissions;
    DROP TABLE IF EXISTS user_roles;
    DROP TABLE IF EXISTS permissions;
    DROP TABLE IF EXISTS roles;
    DROP TABLE IF EXISTS users;
    DROP TABLE IF EXISTS assets;

    -- Core reference tables
    CREATE TABLE assets (
        id              INTEGER PRIMARY KEY,
        hostname        TEXT NOT NULL UNIQUE,
        ip              TEXT NOT NULL UNIQUE,
        os              TEXT NOT NULL,
        business_unit   TEXT NOT NULL
    );

    CREATE TABLE users (
        id          INTEGER PRIMARY KEY,
        username    TEXT NOT NULL UNIQUE,
        dept        TEXT NOT NULL,
        disabled    INTEGER NOT NULL DEFAULT 0 -- 0=false, 1=true
    );

    -- Simple RBAC schema (for privilege-style practice)
    CREATE TABLE roles (
        id          INTEGER PRIMARY KEY,
        name        TEXT NOT NULL UNIQUE
    );

    CREATE TABLE permissions (
        id          INTEGER PRIMARY KEY,
        resource    TEXT NOT NULL,      -- e.g., "alerts", "flows", "dns"
        action      TEXT NOT NULL       -- e.g., "read", "write", "triage"
    );

    CREATE TABLE user_roles (
        user_id     INTEGER NOT NULL,
        role_id     INTEGER NOT NULL,
        PRIMARY KEY (user_id, role_id),
        FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE,
        FOREIGN KEY (role_id) REFERENCES roles(id) ON DELETE CASCADE
    );

    CREATE TABLE role_permissions (
        role_id     INTEGER NOT NULL,
        perm_id     INTEGER NOT NULL,
        PRIMARY KEY (role_id, perm_id),
        FOREIGN KEY (role_id) REFERENCES roles(id) ON DELETE CASCADE,
        FOREIGN KEY (perm_id) REFERENCES permissions(id) ON DELETE CASCADE
    );

    -- Event/Log tables
    CREATE TABLE auth_logs (
        id          INTEGER PRIMARY KEY,
        ts          TEXT NOT NULL,
        user_id     INTEGER NOT NULL,
        asset_id    INTEGER NOT NULL,
        src_ip      TEXT NOT NULL,
        method      TEXT NOT NULL,          -- password, mfa, ssh_key
        success     INTEGER NOT NULL,       -- 0/1
        reason      TEXT,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (asset_id) REFERENCES assets(id)
    );

    CREATE TABLE process_events (
        id              INTEGER PRIMARY KEY,
        ts              TEXT NOT NULL,
        asset_id        INTEGER NOT NULL,
        user_id         INTEGER,
        process_name    TEXT NOT NULL,
        pid             INTEGER NOT NULL,
        parent_pid      INTEGER,
        sha256          TEXT,
        signed          INTEGER,            -- 0/1/NULL
        FOREIGN KEY (asset_id) REFERENCES assets(id),
        FOREIGN KEY (user_id)  REFERENCES users(id)
    );

    CREATE TABLE dns_logs (
        id          INTEGER PRIMARY KEY,
        ts          TEXT NOT NULL,
        asset_id    INTEGER NOT NULL,
        query       TEXT NOT NULL,
        qtype       TEXT NOT NULL,
        response_ip TEXT,
        FOREIGN KEY (asset_id) REFERENCES assets(id)
    );

    CREATE TABLE network_flows (
        id          INTEGER PRIMARY KEY,
        ts          TEXT NOT NULL,
        asset_id    INTEGER NOT NULL,
        src_ip      TEXT NOT NULL,
        dst_ip      TEXT NOT NULL,
        dst_port    INTEGER NOT NULL,
        proto       TEXT NOT NULL,      -- TCP/UDP
        bytes_tx    INTEGER NOT NULL,
        bytes_rx    INTEGER NOT NULL,
        action      TEXT NOT NULL,      -- allowed/blocked
        FOREIGN KEY (asset_id) REFERENCES assets(id)
    );

    -- Detection and response
    CREATE TABLE alerts (
        id          INTEGER PRIMARY KEY,
        ts          TEXT NOT NULL,
        rule_name   TEXT NOT NULL,
        severity    TEXT NOT NULL,      -- low/medium/high/critical
        source      TEXT NOT NULL,      -- EDR/SIEM/IDS
        asset_id    INTEGER,
        user_id     INTEGER,
        status      TEXT NOT NULL,      -- new/in_progress/closed
        FOREIGN KEY (asset_id) REFERENCES assets(id),
        FOREIGN KEY (user_id)  REFERENCES users(id)
    );

    CREATE TABLE iocs (
        id          INTEGER PRIMARY KEY,
        type        TEXT NOT NULL,      -- ip, domain, hash
        value       TEXT NOT NULL UNIQUE,
        first_seen  TEXT NOT NULL
    );

    CREATE TABLE alert_ioc (
        alert_id    INTEGER NOT NULL,
        ioc_id      INTEGER NOT NULL,
        PRIMARY KEY (alert_id, ioc_id),
        FOREIGN KEY (alert_id) REFERENCES alerts(id) ON DELETE CASCADE,
        FOREIGN KEY (ioc_id)   REFERENCES iocs(id)   ON DELETE CASCADE
    );

    CREATE TABLE casebook (
        id          INTEGER PRIMARY KEY,
        opened_ts   TEXT NOT NULL,
        owner       TEXT NOT NULL,
        status      TEXT NOT NULL,      -- open/investigating/contained/closed
        priority    TEXT NOT NULL       -- P1/P2/P3
    );

    CREATE TABLE case_alert (
        case_id     INTEGER NOT NULL,
        alert_id    INTEGER NOT NULL,
        PRIMARY KEY (case_id, alert_id),
        FOREIGN KEY (case_id) REFERENCES casebook(id) ON DELETE CASCADE,
        FOREIGN KEY (alert_id) REFERENCES alerts(id) ON DELETE CASCADE
    );

    -- Vulnerability findings (great for joins & aggregates)
    CREATE TABLE vuln_findings (
        id          INTEGER PRIMARY KEY,
        asset_id    INTEGER NOT NULL,
        plugin_id   TEXT NOT NULL,
        cve         TEXT,
        severity    TEXT NOT NULL,      -- low/medium/high/critical
        found_ts    TEXT NOT NULL,
        fixed_ts    TEXT,
        FOREIGN KEY (asset_id) REFERENCES assets(id)
    );

    -- Useful indexes
    CREATE INDEX idx_auth_ts ON auth_logs(ts);
    CREATE INDEX idx_auth_user ON auth_logs(user_id);
    CREATE INDEX idx_auth_asset ON auth_logs(asset_id);

    CREATE INDEX idx_proc_ts ON process_events(ts);
    CREATE INDEX idx_proc_asset ON process_events(asset_id);

    CREATE INDEX idx_dns_ts ON dns_logs(ts);
    CREATE INDEX idx_dns_query ON dns_logs(query);

    CREATE INDEX idx_flow_ts ON network_flows(ts);
    CREATE INDEX idx_flow_dst ON network_flows(dst_ip, dst_port);
    CREATE INDEX idx_flow_asset ON network_flows(asset_id);

    CREATE INDEX idx_alert_ts ON alerts(ts);
    CREATE INDEX idx_alert_asset ON alerts(asset_id);
    CREATE INDEX idx_alert_user ON alerts(user_id);

    CREATE INDEX idx_vuln_asset ON vuln_findings(asset_id);
    """)
    conn.commit()

def seed_reference(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()

    assets = [
        (1, "ws-alice", "10.0.1.10", "Windows 11", "Sales"),
        (2, "ws-bob",   "10.0.1.11", "Windows 11", "Finance"),
        (3, "srv-edr",  "10.0.10.5", "Linux",      "Security"),
        (4, "srv-web",  "10.0.20.9", "Linux",      "IT"),
        (5, "ws-char",  "10.0.1.12", "Windows 11", "HR"),
    ]
    users = [
        (1, "alice",   "Sales",   0),
        (2, "bob",     "Finance", 0),
        (3, "charlie", "HR",      0),
        (4, "svc_edr", "Security",0),
        (5, "disabled","IT",      1),
    ]

    roles = [(1, "SOC_Analyst"), (2, "IR_Lead"), (3, "Viewer")]
    perms = [
        (1, "alerts", "read"), (2, "alerts", "triage"), (3, "alerts", "close"),
        (4, "flows", "read"),  (5, "dns", "read"),      (6, "cases", "manage")
    ]
    user_roles = [(1,1), (1,3), (2,3), (3,3), (4,2)]  # alice: SOC+Viewer, svc_edr: IR_Lead
    role_perms = [
        (1,1),(1,2),(1,4),(1,5),      # SOC_Analyst: read/triage alerts, read flows/dns
        (2,1),(2,2),(2,3),(2,6),      # IR_Lead: full alerts + manage cases
        (3,1),(3,4),(3,5)             # Viewer: read alerts/flows/dns
    ]

    cur.executemany("INSERT INTO assets VALUES (?,?,?,?,?)", assets)
    cur.executemany("INSERT INTO users  VALUES (?,?,?,?)", users)
    cur.executemany("INSERT INTO roles  VALUES (?,?)", roles)
    cur.executemany("INSERT INTO permissions VALUES (?,?,?)", perms)
    cur.executemany("INSERT INTO user_roles VALUES (?,?)", user_roles)
    cur.executemany("INSERT INTO role_permissions VALUES (?,?)", role_perms)
    conn.commit()

def seed_events(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()
    base = datetime(2025, 8, 1, 9, 0, 0)

    # Auth logs (mix success/fail, mfa/password)
    auth_rows = []
    auth_id = 1
    for day in range(0, 4):
        for hour in [9, 12, 15, 23]:
            ts = (base + timedelta(days=day, hours=hour)).isoformat()
            for user_id, asset_id in [(1,1),(2,2),(3,5)]:
                for attempt in range(0, 3):
                    success = 1 if attempt == 0 else 0
                    method = random.choice(["password","mfa","ssh_key"])
                    reason = None if success == 1 else random.choice(["bad_password","locked","mfa_timeout"])
                    src_ip = f"192.168.{day}.{50+attempt}"
                    auth_rows.append((auth_id, ts, user_id, asset_id, src_ip, method, success, reason))
                    auth_id += 1
    cur.executemany("INSERT INTO auth_logs VALUES (?,?,?,?,?,?,?,?)", auth_rows)

    # Process events
    procs = []
    pid = 1000
    for asset_id in [1,2,4]:
        for i in range(10):
            ts = (base + timedelta(hours=i)).isoformat()
            user_id = random.choice([1,2,3,None])
            pname = random.choice(["powershell.exe","cmd.exe","chrome.exe","sshd","nginx","python"])
            ppid = pid - 1 if pid > 1000 else None
            sha = None if random.random() < 0.7 else f"{random.getrandbits(128):032x}"
            signed = None if "exe" not in pname else random.choice([0,1])
            procs.append((None, ts, asset_id, user_id, pname, pid, ppid, sha, signed))
            pid += 1
    cur.executemany("""
        INSERT INTO process_events(id, ts, asset_id, user_id, process_name, pid, parent_pid, sha256, signed)
        VALUES (?,?,?,?,?,?,?,?,?)
    """, procs)

    # DNS logs
    domains = ["login.microsoftonline.com","update.windows.com","malware.badco.ru","api.company.local","cdn.vendor.com"]
    dns_rows = []
    did = 1
    for asset_id in [1,2,3,4,5]:
        for i in range(15):
            ts = (base + timedelta(minutes=15*i)).isoformat()
            q = random.choice(domains)
            qtype = random.choice(["A","AAAA"])
            rip = "93.184.216.34" if q != "malware.badco.ru" else "203.0.113.200"
            dns_rows.append((did, ts, asset_id, q, qtype, rip))
            did += 1
    cur.executemany("INSERT INTO dns_logs VALUES (?,?,?,?,?,?)", dns_rows)

    # Network flows
    flows = []
    fid = 1
    for asset_id in [1,2,4,5]:
        for i in range(20):
            ts = (base + timedelta(minutes=10*i)).isoformat()
            src = f"10.0.1.{10+asset_id}"
            dst = random.choice(["52.218.101.1","20.190.128.1","203.0.113.200","10.0.20.9"])
            dport = random.choice([80,443,22,445,3389])
            proto = random.choice(["TCP","UDP"])
            tx = random.randint(500, 200000)
            rx = random.randint(500, 200000)
            action = "blocked" if dst == "203.0.113.200" and dport in [22,445] else "allowed"
            flows.append((fid, ts, asset_id, src, dst, dport, proto, tx, rx, action))
            fid += 1
    cur.executemany("INSERT INTO network_flows VALUES (?,?,?,?,?,?,?,?,?,?)", flows)

    # IOCs + Alerts
    iocs = [
        (1, "ip", "203.0.113.200", (base + timedelta(days=-10)).isoformat()),
        (2, "domain", "malware.badco.ru", (base + timedelta(days=-8)).isoformat()),
        (3, "hash", "deadbeef"*8, (base + timedelta(days=-5)).isoformat()),
    ]
    cur.executemany("INSERT INTO iocs VALUES (?,?,?,?)", iocs)

    alerts = [
        (1, (base + timedelta(days=1, hours=10)).isoformat(), "Multiple Failed Logins", "medium", "SIEM", 1, 1, "new"),
        (2, (base + timedelta(days=1, hours=11)).isoformat(), "Suspicious DNS Query", "high", "IDS", 2, None, "in_progress"),
        (3, (base + timedelta(days=2, hours=9)).isoformat(),  "Outbound to Known Bad IP", "critical", "IDS", 4, None, "new"),
        (4, (base + timedelta(days=2, hours=12)).isoformat(), "Unsigned Powershell", "high", "EDR", 1, 1, "new"),
    ]
    cur.executemany("INSERT INTO alerts VALUES (?,?,?,?,?,?,?,?)", alerts)

    alert_ioc = [
        (2, 2),   # Suspicious DNS → bad domain
        (3, 1),   # Outbound to bad IP
        (4, 3),   # Unsigned binary hash (example)
    ]
    cur.executemany("INSERT INTO alert_ioc VALUES (?,?)", alert_ioc)

    # Cases
    cases = [
        (1, (base + timedelta(days=1, hours=11, minutes=30)).isoformat(), "alice", "open", "P2"),
        (2, (base + timedelta(days=2, hours=9, minutes=30)).isoformat(),  "svc_edr", "investigating", "P1"),
    ]
    cur.executemany("INSERT INTO casebook VALUES (?,?,?,?,?)", cases)
    cur.executemany("INSERT INTO case_alert VALUES (?,?)", [(1,2),(2,3)])

    # Vuln findings
    vulns = []
    vid = 1
    for asset_id in [1,2,3,4,5]:
        for sev in ["low","medium","high","critical"]:
            found = (base + timedelta(days=-random.randint(1,20))).isoformat()
            fixed = None if sev in ["high","critical"] and random.random() < 0.7 else (base + timedelta(days=-random.randint(0,5))).isoformat()
            vulns.append((vid, asset_id, f"plg-{asset_id}-{sev}", f"CVE-2024-{random.randint(1000,9999)}", sev, found, fixed))
            vid += 1
    cur.executemany("INSERT INTO vuln_findings VALUES (?,?,?,?,?,?,?)", vulns)

    conn.commit()

def example_queries(conn: sqlite3.Connection) -> None:
    cur = conn.cursor()

    print("\n-- JOIN: Alerts enriched with asset + user --")
    for row in cur.execute("""
        SELECT a.id, a.ts, a.rule_name, a.severity,
               asst.hostname, u.username, a.status
        FROM alerts a
        LEFT JOIN assets asst ON a.asset_id = asst.id
        LEFT JOIN users  u    ON a.user_id  = u.id
        ORDER BY a.ts;
    """):
        print(row)

    print("\n-- SUBQUERY: Assets that talked to an IOC IP --")
    for row in cur.execute("""
        SELECT DISTINCT asst.hostname
        FROM network_flows f
        JOIN iocs i ON i.type='ip' AND f.dst_ip = i.value
        JOIN assets asst ON asst.id = f.asset_id
        ORDER BY 1;
    """):
        print(row)

    print("\n-- AGGREGATE: Top DNS queries by count --")
    for row in cur.execute("""
        SELECT query, COUNT(*) as cnt
        FROM dns_logs
        GROUP BY query
        ORDER BY cnt DESC, query ASC
        LIMIT 5;
    """):
        print(row)

    print("\n-- CORRELATED SUBQUERY: Users with above-average failed logins --")
    for row in cur.execute("""
        SELECT u.username, 
               SUM(CASE WHEN a.success=0 THEN 1 ELSE 0 END) AS fails
        FROM users u
        JOIN auth_logs a ON a.user_id = u.id
        GROUP BY u.username
        HAVING fails > (
            SELECT AVG(fail_cnt) FROM (
                SELECT SUM(CASE WHEN success=0 THEN 1 ELSE 0 END) AS fail_cnt
                FROM auth_logs
                GROUP BY user_id
            )
        )
        ORDER BY fails DESC;
    """):
        print(row)

    print("\n-- INDEX PRACTICE: flows to 203.0.113.200:445 (should be fast) --")
    for row in cur.execute("""
        SELECT ts, asset_id, src_ip, dst_ip, dst_port, action
        FROM network_flows
        WHERE dst_ip = '203.0.113.200' AND dst_port = 445
        ORDER BY ts
        LIMIT 10;
    """):
        print(row)

    print("\n-- RBAC: What permissions does 'alice' effectively have? --")
    for row in cur.execute("""
        SELECT DISTINCT p.resource, p.action
        FROM users u
        JOIN user_roles ur ON ur.user_id = u.id
        JOIN role_permissions rp ON rp.role_id = ur.role_id
        JOIN permissions p ON p.id = rp.perm_id
        WHERE u.username = 'alice'
        ORDER BY p.resource, p.action;
    """):
        print(row)

    print("\n-- TRANSACTION: Close all alerts in Case 1 atomically --")
    try:
        cur.execute("BEGIN;")
        cur.execute("""
            UPDATE alerts
            SET status = 'closed'
            WHERE id IN (SELECT alert_id FROM case_alert WHERE case_id = 1)
        """)
        cur.execute("""
            UPDATE casebook SET status='closed'
            WHERE id = 1
        """)
        conn.commit()
        print("Transaction committed.")
    except Exception as e:
        conn.rollback()
        print("Transaction rolled back:", e)

    print("\n-- Validate Case 1 closure --")
    for row in cur.execute("""
        SELECT c.id, c.status, GROUP_CONCAT(a.status)
        FROM casebook c
        LEFT JOIN case_alert ca ON ca.case_id = c.id
        LEFT JOIN alerts a ON a.id = ca.alert_id
        WHERE c.id = 1
        GROUP BY c.id, c.status;
    """):
        print(row)

# --------- Run everything ----------
conn = connect_db(DB_PATH)
reset_schema(conn)
seed_reference(conn)
seed_events(conn)
example_queries(conn)
conn.close()

print("\n[OK] 'sec_logs.db' ready. Try queries against it!")



-- JOIN: Alerts enriched with asset + user --
(1, '2025-08-02T19:00:00', 'Multiple Failed Logins', 'medium', 'ws-alice', 'alice', 'new')
(2, '2025-08-02T20:00:00', 'Suspicious DNS Query', 'high', 'ws-bob', None, 'in_progress')
(3, '2025-08-03T18:00:00', 'Outbound to Known Bad IP', 'critical', 'srv-web', None, 'new')
(4, '2025-08-03T21:00:00', 'Unsigned Powershell', 'high', 'ws-alice', 'alice', 'new')

-- SUBQUERY: Assets that talked to an IOC IP --
('srv-web',)
('ws-alice',)
('ws-bob',)
('ws-char',)

-- AGGREGATE: Top DNS queries by count --
('malware.badco.ru', 21)
('api.company.local', 14)
('login.microsoftonline.com', 14)
('cdn.vendor.com', 13)
('update.windows.com', 13)

-- CORRELATED SUBQUERY: Users with above-average failed logins --

-- INDEX PRACTICE: flows to 203.0.113.200:445 (should be fast) --
('2025-08-01T09:50:00', 2, '10.0.1.12', '203.0.113.200', 445, 'blocked')
('2025-08-01T10:00:00', 5, '10.0.1.15', '203.0.113.200', 445, 'blocked')
('2025-08-01T10:40:00', 5, '10.0.1.1